In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def get_tree_polygon(x, y, deg):
    """Create tree polygon at position (x, y) with rotation deg"""
    base_poly = Polygon(zip(TX, TY))
    rotated = affinity.rotate(base_poly, deg, origin=(0, 0))
    return affinity.translate(rotated, xoff=x, yoff=y)

def calculate_score(csv_path):
    """Calculate total score for a submission"""
    df = pd.read_csv(csv_path)
    
    # Parse values (remove 's' prefix)
    df['x_val'] = df['x'].str[1:].astype(float)
    df['y_val'] = df['y'].str[1:].astype(float)
    df['deg_val'] = df['deg'].str[1:].astype(float)
    df['n'] = df['id'].str.split('_').str[0].astype(int)
    
    total_score = 0
    scores_by_n = {}
    
    for n in range(1, 201):
        group = df[df['n'] == n]
        if len(group) == 0:
            continue
            
        # Get all tree polygons
        all_coords = []
        for _, row in group.iterrows():
            poly = get_tree_polygon(row['x_val'], row['y_val'], row['deg_val'])
            coords = np.array(poly.exterior.coords)
            all_coords.append(coords)
        
        all_coords = np.vstack(all_coords)
        
        # Calculate bounding box side
        x_range = all_coords[:, 0].max() - all_coords[:, 0].min()
        y_range = all_coords[:, 1].max() - all_coords[:, 1].min()
        side = max(x_range, y_range)
        
        score_n = side**2 / n
        scores_by_n[n] = {'side': side, 'score': score_n}
        total_score += score_n
    
    return total_score, scores_by_n

print("Calculating score for pre-optimized submission...")
csv_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
total, by_n = calculate_score(csv_path)
print(f"Total score: {total:.6f}")
print(f"\nTop 10 highest contributing N values:")
sorted_by_score = sorted(by_n.items(), key=lambda x: x[1]['score'], reverse=True)[:10]
for n, data in sorted_by_score:
    print(f"  N={n}: side={data['side']:.6f}, score={data['score']:.6f}")